In [2]:
import numpy as np
import random
import time

Wskaźnik uwarunkowania macierzy

In [3]:
def norm(A):
    n = len(A)
    return max(sum(A[i][j] for j in range(n)) for i in range(n))

In [4]:
def conditioning_factor(A):
    A_inv = np.linalg.inv(A)
    return norm(A_inv) * norm(A)

Gaussian Elimination

![title](img_vsc/img01.png)

![title](img_vsc/img02.png)

In [35]:
def gaussian_elimination(A, B, data_type):
    
    n = A.shape[0]
    
    C = np.hstack((A, B.reshape(n, 1)))


    for i in range(n):
        for j in range(i + 1, n):
            ratio = C[j][i] / C[i][i]
            C[j] = C[j] - ratio * C[i]

    X = np.zeros(n, dtype=data_type)

    X[n - 1] = C[n - 1][n] / C[n - 1][n - 1] # We now have n + 1 columns

    for i  in range(n - 2 , -1, -1):
        X[i] = (C[i][n] - sum(X[i+1:n]*C[i][i+1:n])) / C[i][i]
        
    return X

In [69]:
class GaussTest():
    
    def __init__(self, get_matrix_function):

        self.get_matrix = get_matrix_function

    def random_vector(self, n):
        
        V = []
        
        for _ in range(n):
            V.append( -1 if random.random() < 0.5 else 1)
            
        return V
    
        
    def solve(self, n, data_type):
        
        A = self.get_matrix(n)   
        X = self.random_vector(n)
        
        A = np.array(A, dtype=data_type).reshape(n, n)
        X = np.array(X, dtype=data_type).reshape(n)
        
        B = A @ X
        
        start = time.time()    

        X_approx = gaussian_elimination(A, B, data_type)

        end = time.time()
        
        if data_type == np.float32 or data_type == np.float64:
            return X, X_approx, end - start, conditioning_factor(A)
        else:
            return X, X_approx, end - start, None
        
    
    def solve_n(self, n_min, n_max, data_type):
        """
        Performs tests for n_min, n_min + 1, ..., n_max and given data_type.
        Returns a list of results
        """
        results = []

        for n in range(n_min, n_max + 1):

            X, X_approx, execution_time, conditioning = self.solve(n, data_type)
            
            results.append([n, execution_time, conditioning, np.linalg.norm(X - X_approx)])
            
        return results

In [78]:
def zad1_matrix(n):
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
        
    for i in range(1, n + 1):
        for j in range(1, n + 1):
                
            if i == 1:
                A[i - 1][j - 1] = 1
            else:
                A[i - 1][j - 1] =  1 / (i + j - 1)
                    
    return A

Examples

In [79]:
zad1 = GaussTest(zad1_matrix)

X, X_approx, exec_time, cf = zad1.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[ 1.  1.  1. -1.  1.] [ 0.99988645  1.0010474   0.9970335  -0.99668515  0.99871784] 0.0003409385681152344 28023.636474609375


In [80]:
zad1 = GaussTest(zad1_matrix)

X, X_approx, exec_time, cf = zad1.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[ 1. -1. -1. -1.  1.] [ 1. -1. -1. -1.  1.] 0.00040435791015625 28000.000000047658


Tests

In [84]:
zad1 = GaussTest(zad1_matrix)
zad1_results_float32 = zad1.solve_n(3, 200, np.float32)

In [86]:
for n, exec_time, conditioning, error in zad1_results_float32: # no conditioning since its float16
    print(n, exec_time,conditioning, error)

3 0.00029397010803222656 216.00041484832764 0.0
4 0.00016164779663085938 2880.032928466797 0.00020188167
5 0.0002377033233642578 28023.636474609375 0.011283298
6 0.00025391578674316406 232606.4384765625 0.023149196
7 0.0003943443298339844 2898755.69921875 5.187338
8 0.0004172325134277344 1316895.181640625 13.110759
9 0.0017940998077392578 14008956.3984375 24.922
10 0.0009570121765136719 4207927.08984375 10.34897
11 0.0006666183471679688 6356634.32421875 40.820114
12 0.0007817745208740234 2712885.1083984375 45.046307
13 0.0010135173797607422 2723394.9462890625 20.490215
14 0.001104593276977539 3962083.2719726562 2.666362
15 0.0012927055358886719 20024231.909179688 27.486298
16 0.0013625621795654297 19575512.140625 29.425459
17 0.0015506744384765625 25471486.899414062 9.945443
18 0.0017216205596923828 16330873.974609375 36.650734
19 0.002938985824584961 7791152.4873046875 11.919175
20 0.002195596694946289 5265721.1962890625 12.537539
21 0.002424478530883789 9564600.394042969 33.347996
22

In [87]:
zad1 = GaussTest(zad1_matrix)
zad1_results_float64 = zad1.solve_n(3, 200, np.float64)

In [88]:
for n, exec_time, conditioning, error in zad1_results_float64:
    print(n, exec_time, conditioning, error)

3 0.00022530555725097656 215.9999999999994 3.373781268209917e-14
4 0.00018477439880371094 2880.0000000004075 2.7910574051884874e-15
5 0.00019121170043945312 28000.000000047658 9.229383358318729e-12
6 0.00034689903259277344 226799.99999259738 4.296958750573517e-11
7 0.00041747093200683594 1629935.9973776676 3.115806677979756e-09
8 0.0004971027374267578 12862080.919548035 3.2406420799920784e-08
9 0.0005192756652832031 112000171.55565262 2.6903023309017226e-06
10 0.0006120204925537109 884143809.3249512 7.153090834690557e-05
11 0.0028083324432373047 6473791750.550781 0.00803524169485115
12 0.0007026195526123047 44079388491.0 0.4130110804899155
13 0.0007948875427246094 134767071829.0 9.61140604272373
14 0.0008013248443603516 245922411521.5 11.130829088581422
15 0.0008993148803710938 173330914155.0 3.680078803831936
16 0.0010395050048828125 108497229744.0 4.063942266345099
17 0.0014700889587402344 304548539524.0 17.37848639991954
18 0.0017993450164794922 118749314327040.0 5.244205370794849
1

In [37]:
def zad2_matrix(n):
    
    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]
    
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j >= i:
                A[i - 1][j - 1] = 2 * i / j
            else:
                A[i - 1][j - 1] = A[j - 1][i - 1]
                
    return A

Examples

In [38]:
zad2 = GaussTest(zad2_matrix)

X, X_approx, exec_time, cf = zad2.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1.  1.  1. -1. -1.] [-1.          1.0000001   0.99999994 -1.         -1.        ] 0.0009829998016357422 2.233333435654641


In [39]:
zad2 = GaussTest(zad2_matrix)

X, X_approx, exec_time, cf = zad2.solve(5, np.float64)
print(X, X_approx, cf)

[ 1. -1.  1. -1. -1.] [ 1. -1.  1. -1. -1.] 2.233333333333334


Tests

In [40]:
zad2 = GaussTest(zad2_matrix)
zad2_results_float16 = zad2.solve_n(3, 300, np.float16)

In [41]:
for n, exec_time, conditioning, error in zad2_results_float16:
    print(n, exec_time, conditioning, error)

3 0.00027441978454589844 None 0.002392
4 0.0001652240753173828 None 0.003275
5 0.0003161430358886719 None 0.00604
6 0.00021529197692871094 None 0.00858
7 0.0002830028533935547 None 0.014694
8 0.00028634071350097656 None 0.02208
9 0.0003504753112792969 None 0.01211
10 0.0004248619079589844 None 0.02333
11 0.0005049705505371094 None 0.01817
12 0.0006086826324462891 None 0.0414
13 0.0006952285766601562 None 0.03558
14 0.000820159912109375 None 0.03928
15 0.0009338855743408203 None 0.04855
16 0.0010483264923095703 None 0.0771
17 0.0014226436614990234 None 0.1394
18 0.002627134323120117 None 0.0767
19 0.0018808841705322266 None 0.0743
20 0.001621246337890625 None 0.125
21 0.0017898082733154297 None 0.128
22 0.0019516944885253906 None 0.1323
23 0.0021407604217529297 None 0.2532
24 0.0023348331451416016 None 0.241
25 0.003041982650756836 None 0.279
26 0.002896547317504883 None 0.2163
27 0.0029888153076171875 None 0.2725
28 0.0032143592834472656 None 0.2476
29 0.003566741943359375 None 0.5537


In [30]:
zad2 = GaussTest(zad2_matrix)
zad2_results_float32 = zad2.solve_n(3, 300, np.float32)

In [32]:
for n, exec_time, conditioning, error in zad2_results_float32:
    print(n, exec_time, conditioning, error)

3 0.00024580955505371094 1.4444445007377202 0.0
4 0.0001556873321533203 1.8333334078391397 0.0
5 0.0004482269287109375 2.233333435654641 9.0198193e-07
6 0.001310586929321289 2.6444445444477935 1.0876827e-06
7 0.00031495094299316406 3.031746150009214 1.8876896e-06
8 0.0005247592926025391 3.4484128290935177 2.2166198e-06
9 0.0005388259887695312 3.849206492777855 4.0692994e-06
10 0.0018410682678222656 4.2492065205933525 1.5859764e-06
11 0.0007135868072509766 4.6594277916181355 5.1508896e-06
12 0.0009210109710693359 5.055219045933653 6.3868815e-06
13 0.0006496906280517578 5.465475483699896 3.6177614e-06
14 0.0006728172302246094 5.868897985485124 5.6246704e-06
15 0.0007688999176025391 6.268898013300622 1.4662379e-05
16 0.0012826919555664062 6.678405180923292 9.312451e-06
17 0.0011110305786132812 7.077618273425633 1.695561e-05
18 0.0011310577392578125 7.485025688559575 6.9415746e-06
19 0.0012967586517333984 7.889565429773881 2.5662135e-05
20 0.0015025138854980469 8.28956545758938 2.197076e-0

In [31]:
zad2 = GaussTest(zad2_matrix)
zad2_results_float64 = zad2.solve_n(3, 300, np.float64)

In [27]:
for n, exec_time, conditioning, error in zad2_results_float64:
    print(n, exec_time, conditioning, error)

3 0.0001926422119140625 1.4444444444444444 0.0
4 0.00012493133544921875 1.8333333333333341 6.280369834735101e-16
5 0.00017786026000976562 2.233333333333334 2.482534153247273e-16
6 0.00020813941955566406 2.6444444444444435 1.7055569729570692e-15
7 0.0002410411834716797 3.0317460317460325 6.753223014464259e-16
8 0.00027561187744140625 3.4484126984126986 1.631687946612622e-15
9 0.00026726722717285156 3.849206349206351 2.085919684541963e-15
10 0.0003204345703125 4.24920634920635 7.666168367507028e-15
11 0.0005581378936767578 4.659427609427611 3.784541690934187e-15
12 0.001087188720703125 5.055218855218855 9.91150011871584e-15
13 0.00067138671875 5.465475265475266 1.2443416112664228e-14
14 0.0007512569427490234 5.868897768897767 1.7196992696293468e-14
15 0.0008893013000488281 6.268897768897766 1.7274941830720587e-14
16 0.0009856224060058594 6.678404928404934 1.5307797578540578e-14
17 0.0010378360748291016 7.07761797026504 2.314275806464067e-14
18 0.0012192726135253906 7.485025377672442 1.16

In [18]:
def zad3a_matrix(n):

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):

            if j == i:
                A[i - 1][j - 1] = k
            elif j == i + 1:
                A[i - 1][j - 1] = 1 / (i + m)
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = k / (i + m + 1)

    return A

Examples

In [93]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float16)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1.     -1.      0.9995  0.9995 -1.    ] None


In [19]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1.         -0.99999994  1.          1.         -1.0000001 ] 0.0005900859832763672 1.159485363403662


In [20]:
zad3a = GaussTest(zad3a_matrix)

X, X_approx, exec_time, cf = zad3a.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1. -1.  1.  1. -1.] 0.0005328655242919922 1.1594853983864615


In [21]:
def zad3b_matrix(n):

    k = 6
    m = 3

    A = [[0 for j in range(1, n + 1)] for i in range(1, n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            
            if j == i:
                A[i - 1][j - 1] = -m * i - k
            elif j == i + 1:
                A[i - 1][j - 1] = i
            elif i > 1 and j == i - 1:
                A[i - 1][j - 1] = m / i

    return A

Examples

In [22]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf = zad3b.solve(5, np.float16)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1. -1.  1.  1. -1.] 0.0013349056243896484 None


In [23]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf= zad3b.solve(5, np.float32)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1. -1.  1.  1. -1.] 0.0009522438049316406 0.3970123413610054


In [24]:
zad3b = GaussTest(zad3b_matrix)

X, X_approx, exec_time, cf = zad3b.solve(5, np.float64)
print(X, X_approx, exec_time, cf)

[-1. -1.  1.  1. -1.] [-1. -1.  1.  1. -1.] 0.0002276897430419922 0.3970123323338485


Algorytm Thomasa dla macierzy trójdiagonalnej

In [99]:
def thomas_algorithm(A, B, C, D):
    
    N = len(B)
    
    C[0] /= B[0]
    D[0] /= B[0]
    
    for i in range(1, N - 1):
        
        common_coefficient = B[i] - A[i - 1] * C[i - 1]
        
        C[i] /= common_coefficient
        D[i] = (D[i] - A[i - 1] * D[i - 1])  / common_coefficient
        
    # Since loop goes only to n - 1, we need to calculate the last element of D
    D[N - 1] = (D[N - 1] - A[N - 2] * D[N - 2]) / (B[N - 1] - A[N - 2] * C[N - 2])
    
    # Backward substitution
    X = [0] * N
    X[-1] = D[-1]
    
    for i in range(N - 2, -1, -1):
        X[i] = D[i] - C[i] * X[i + 1]
        
    return X    

In [100]:
A = [3, 1, 3]
B = [10, 10, 7, 4]
C = [2, 4, 5]
D = [3, 4, 5, 6]

X = thomas_algorithm(A, B, C, D)
print(X)


A = np.array([[10,2,0,0],[3,10,4,0],[0,1,7,5],[0,0,3,4]],dtype=float)   
a = np.array([3.,1,3]) 
b = np.array([10.,10.,7.,4.])
c = np.array([2.,4.,5.])
d = np.array([3,4,5,6.])
print(np.linalg.solve(A, d))

[0.14877589453860635, 0.7561205273069682, -1.0018832391713752, 2.2514124293785316]
[ 0.14877589  0.75612053 -1.00188324  2.25141243]
